In [0]:

with L1 as (
  select TradingDate, TradingPeriodNumber, MegawattsByGenerationType/2 as MWh,
  floor((datediff(TradingDate, date_trunc('year', TradingDate)) / 7) + 1) as WeekOfYear
  from datahub.wholesale.emissionsbygenerationtype
  where TradingDate >= '2024-01-01' and GenerationType = 'Coal'
), Coal as (
  select year(TradingDate) as `Year`, case when WeekOfYear > 52 then 52 else WeekOfYear end as `Week`, round(sum(MWh),3) as MWh
  from L1 group by 1,2
), L2 as (
  select TradingDate, TradingPeriodNumber, sum(KilowattHours/1000) as MWh,
  floor((datediff(TradingDate, date_trunc('year', TradingDate)) / 7) + 1) as WeekOfYear
  from datahub.wholesale.reconciledinjectionandofftake
  where TradingDate >= '2024-01-01' and PointOfConnectionCode = 'HLY2201'
  and TraderCode = 'GENE' and FlowDirection = 'Injection'
  group by 1,2
), Total as (
select year(TradingDate) as `Year`, case when WeekOfYear > 52 then 52 else WeekOfYear end as `Week`, round(sum(MWh),3) as MWh
from L2 group by 1,2
)
select Total.Year, Total.Week, Total.MWh as TotalMWh, Coal.MWh as CoalMWh, Total.MWh - Coal.MWh as GasMwh 
from Total inner join Coal using (`Year`,`Week`)
order by 1, 2

In [0]:

with L1 as (
  select TradingDate, sum(MegawattsByGenerationType/2) as MWh
  from datahub.wholesale.emissionsbygenerationtype
  where TradingDate >= '2025-01-01' and GenerationType = 'Coal'
  group by all
), L2 as (
  select TradingDate, sum(KilowattHours/1000) as MWh
  from datahub.wholesale.reconciledinjectionandofftake
  where TradingDate >= '2025-01-01' and PointOfConnectionCode = 'HLY2201'
  and TraderCode = 'GENE' and FlowDirection = 'Injection'
  group by all
)
select L2.TradingDate, L2.MWh as TotalMWh, L1.MWh as CoalMWh, L2.MWh - L1.MWh as GasMwh,
L1.MWh * 10.2 as Coal_GJ, L2.MWh * 7.4 as Gas_GJ
from L2 inner join L1 using (`TradingDate`)
order by 1, 2

In [0]:
select * from ea_prd_legacy.copper.em6_emissionsbygenerationtype